In [ ]:

import pandas as pd
import os
import json

from geopy.geocoders import Nominatim
import folium

In [ ]:
# Using file from Main Reference website
parking_df = pd.read_csv('http://iot.ee.surrey.ac.uk:8080/datasets/parking/aarhus_parking.csv', sep=',')
parking_df.head()

In [ ]:
# update garage code to geolocation search to work
parking_df.loc[parking_df['garagecode'] == 'SCANDCENTER', 'garagecode'] = 'SCANDINAVIAN CENTER'
parking_df.head()

# Process geolocation from garageCodes using GeoPy


In [ ]:

garages = parking_df.groupby("garagecode")
print(garages.count())

garage_codes = garages['garagecode'].unique()
print(garage_codes)


# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")
 
# entering the location name
for row in garage_codes:
    getLoc = loc.geocode(f'{row[0]}, Aarhus, Denmark')
    parking_df.loc[parking_df['garagecode'] == row[0], 'latitude'] = getLoc.latitude
    parking_df.loc[parking_df['garagecode'] == row[0],'longitude'] = getLoc.longitude
    parking_df.loc[parking_df['garagecode'] == row[0],'address'] = getLoc.address
    

In [ ]:
parking_df.head()

In [ ]:
na_rows = parking_df[parking_df.isna().any(axis=1)]
na_rows

In [ ]:
# This is the file we will use in our analyze 
parking_df.to_csv('../data/aarhus_parking_geolocated.csv')

## Georeferencing


In [ ]:
AARHUS_GEOLOCATION = [56.16182966470494, 10.197421860666816]

parking_df = pd.read_csv('../data/aarhus_parking_geolocated.csv', sep=',')
map = folium.Map(location=AARHUS_GEOLOCATION, zoom_start=13)
garage_codes = parking_df.groupby(['garagecode', 'address', 'latitude', 'longitude']).count().reset_index()


garage_codes.head()

In [ ]:
# markers on a map
for index, address in garage_codes.iterrows():
   folium.Marker(
         location=[address['latitude'], address['longitude']],
         popup=address['garagecode'],
      ).add_to(map)

map